In [13]:
from datasets import load_dataset
import pandas as pd

In [3]:
dataset = load_dataset(
    "microsoft/ms_marco",
    "v1.1",
    cache_dir="~/experiment_data/datasets/ms_marco"
)

# Materialize full data to download all now
for split in dataset:
    _ = dataset[split][:]

README.md:   0%|          | 0.00/9.48k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/175M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 10047
    })
    train: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 82326
    })
    test: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 9650
    })
})

In [5]:
df_train = dataset['train'].to_pandas()

In [9]:
df_train.iloc[0]['passages']

{'is_selected': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int32),
 'passage_text': array(["Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. These RBA subsidiaries were involved in bribing overseas officials so that Australia might win lucrative note-printing contracts. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.",
        "The Reserve Bank of Australia (RBA) came into being on 14 January 1960 as Australia 's central bank and banknote issuing authority, when the Reserve Bank Act 1959 removed the central banking functions from the Commonwealth Bank. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees 

In [10]:
is_selected_lengths = df_train['passages'].apply(lambda x: len(x['is_selected']))
min_is_selected = is_selected_lengths.min()
max_is_selected = is_selected_lengths.max()

total_ones = df_train['passages'].apply(lambda x: sum(x['is_selected'])).sum()

passage_text_lengths = df_train['passages'].apply(lambda x: len(x['passage_text']))
min_passage_text = passage_text_lengths.min()
max_passage_text = passage_text_lengths.max()

empty_passages = df_train['passages'].apply(lambda x: any(p == "" for p in x['passage_text']))
num_empty_passages = empty_passages.sum()

print(f"min is_selected length: {min_is_selected}")
print(f"max is_selected length: {max_is_selected}")
print(f"total number of 1's in is_selected: {total_ones}")
print(f"min passage_text length: {min_passage_text}")
print(f"max passage_text length: {max_passage_text}")
print(f"number of entries with empty string in passage_text: {num_empty_passages}")

min is_selected length: 1
max is_selected length: 12
total number of 1's in is_selected: 88523
min passage_text length: 1
max passage_text length: 12
number of entries with empty string in passage_text: 0


In [17]:
def display_df(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
        display(df)

In [18]:
print("Rows where 'passage_text' has only 1 element:")
rows_single_passage = df_train[passage_text_lengths == 1]
display_df(rows_single_passage.head())

print("\nRows where 'is_selected' has more than one click:")
rows_multiple_selected = df_train[df_train['passages'].apply(lambda x: sum(x['is_selected']) > 1)]
display_df(rows_multiple_selected.head())

print("\nRows where 'is_selected' has no clicks:")
rows_multiple_selected = df_train[df_train['passages'].apply(lambda x: sum(x['is_selected']) == 0)]
display_df(rows_multiple_selected.head())

Rows where 'passage_text' has only 1 element:


,answers,passages,query,query_id,query_type,wellFormedAnswers
22019,[$2.11],"{'is_selected': [0], 'passage_text': ['3 days ago. $9.90update. There are 15 Diesel gas price reports in the past 5 days in Emporia, VA. The average Diesel gas price in Emporia, VA is $4.99, which is $2.21 higher than U.S. national average Diesel gas price $2.78. The lowest Diesel gas price is $2.3 of MS 58 Plaza located at 1001 Pleasant Shade Rd (US-58), Emporia, VA 23847. The highest Diesel gas price is $9.9 of Market Drive Texaco Slip-In located at 901 Market Dr'], 'url': ['http://www.gaspricewatch.com/VA-virginia/Emporia/diesel-gas-prices/page-1/2.htm']}",gas price emporia va,41736,numeric,[]
31772,[],"{'is_selected': [0], 'passage_text': ['98. Holland Park Av, Kensington Hilton Hotel, bus stop HD. Afternoon courtesy drop off point only. 17:43. If the first bus at your stop (indicated above) is on a different tour than the one you require, catch the first bus to the next interchange point and change onto your tour there. Last Complete Tours. 1 Blue Route from Victoria stop 1 at 17.00 (Summer) Hours & 16.30 (Winter) Hours.'], 'url': ['http://www.hoponhopoffplus.com/Timetable']}",is hop on hop off bus near doubletree hilton london westminster,51501,description,[]
46390,"[Clean,Pure]","{'is_selected': [1], 'passage_text': ['Aramaic Meaning: The name Zacchaeus is an Aramaic baby name. In Aramaic the meaning of the name Zacchaeus is: Pure. Hebrew Meaning: The name Zacchaeus is a Hebrew baby name. In Hebrew the meaning of the name Zacchaeus is: Clean; pure. In the bible Zacchaeus was a tax collector who became a disciples of Jesus. This Database is Copyright Dogwood Technical Services Inc. 1995.'], 'url': ['http://www.sheknows.com/baby-names/name/zacchaeus']}",what does the name zacchaeus mean,66135,description,[]



Rows where 'is_selected' has more than one click:


answers  \
10                                                                                    [Somatic cells are produced by mitosis and gametes produced by most organisms combine to form a zygote with n pairs of chromosomes.]   
21                                                                                                                                                                                         [An hour to 1 hour 15 minutes.]   
42  [Foxes, cats and birds., A possum (plural form: possums) is any of about 70 small-to medium-sized arboreal marsupial species native to Australia, New Guinea, and Sulawesi (and introduced to New Zealand and China).]   
45                                                                                                                                                                                                          [$50 and $200]   
52                                                                                                                                            [$51,000 per year, $72,800 a year salary of a person with a biology degree.]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              


Rows where 'is_selected' has no clicks:


answers  \
7                                                                                          []   
25                                                                                         []   
161  [It is a sharpened tool used to make holes in wood and soft materials like leather etc.]   
166                                                                                        []   
220                                                                                        []   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               